# 說明

1. 想像中是這種流程?? : 從最上層功能選擇推薦電影 > 想要進階搜尋 > 開始搞這堆限搜
2. 給一制式的格式給使用者輸入回傳(傳一次訊息，不填寫表不限制)

    Ex1 :
    
        年度：2000>

        中文片名：我

        原文片名：I

        國別：美國

        語言：英語

        類型：Romance


    Ex2 :

        年度：2000-2020   !

        中文片名：      我

        原文片名：I

        國別：美國

        語言：英語

        類型：Romance Action
        
    Ex3 :

        年度：

        中文片名：
        
        原文片名：

        國別：臺灣

        語言：
        
        類型：
3. 接收完會處理文字後傳給Recommand_request 去推薦


4. 輸入說明
    1. 年份有3種方向
        * 2000> : 2000年之後的
        * 2000< : 2000年之前的
        * 2020-2022 : 有輸入到減號，會去判斷這兩個時間內的
        
    2. 中文片名、英文片名、國家、語言都是模糊搜尋，有那個"字"出現就會搜到

        ex : 國家輸入"國", 有這個字就會查, 不管美國、法國、英國還甚麼的

    3. 類型可以同時查很多類, 用空白間格(大小寫不拘, Only Eng)

        ex : Action Comedy Thriller, 會找同時符合這3個的

In [40]:
# 動態輸入n次
import sqlite3
import random

class Recommand_System():
    def __init__(self):
        self.conn = sqlite3.connect('Movies.db')
        self.cursor = self.conn.cursor()

    def cleanmsg(self, msg):
        row_msg = msg.strip().split('\n')
        result = {item.split('：')[0]: item.split('：')[1].strip() for item in row_msg}
        return result

    # 推薦
    def random_recommand(self, dict:dict = {}):
        # 匯入使用者限搜

        year = dict.get('年度')
        chinese_title = dict.get('中文片名')
        original_title = dict.get('原文片名')
        country = dict.get('國別')
        language = dict.get('語言')
        genre = dict.get('類型')

        query = "SELECT 年度,中文片名,原文片名,國別,語言,類型 FROM Movies_latest WHERE 1=1"
        params = []

        # 年度搜索
        if year:
            if '-' in year:
                # 年份範圍搜索
                start_year, end_year = year.split('-')
                query += " AND 年度 BETWEEN ? AND ?"
                params.extend([start_year, end_year])
            elif year.endswith('>'):
                # 某年之後搜索
                year_offset = year[:-1]
                query += " AND 年度 >= ?"
                params.append(year_offset)
            elif year.endswith('<'):
                # 某年之前搜索
                year_offset = year[:-1]
                query += " AND 年度 <= ?"
                params.append(year_offset)
            else:
                # 指定某年
                query += " AND 年度 = ?"
                params.append(year)

        # 可模糊搜尋
        if chinese_title:
            query += " AND 中文片名 LIKE '%' || ? || '%'"
            params.append(chinese_title)

        # 可模糊搜尋
        if original_title:
            query += " AND 原文片名 LIKE '%' || ? || '%'"
            params.append(original_title)

        # 可模糊搜尋
        if country:
            query += " AND 國別 LIKE '%' || ? || '%'"
            params.append(country)

        # 可模糊搜尋
        if language:
            query += " AND 語言 LIKE '%' || ? || '%'"
            params.append(language)

        # 可多重搜尋
        if genre:
            genre_keywords = genre.split()
            for keyword in genre_keywords:
                query += " AND 類型 LIKE '%' || ? || '%'"
                params.append(keyword)

        
        self.cursor.execute(query, params)
        results = self.cursor.fetchall()
        self.conn.close()

        if not results: return "無結果"

        # 隨機從限搜下推3個
        # 有重複名稱就從新隨機(英文片名判斷)
        if len(results) > 3:
            while True:
                random_results = random.sample(results, 3)
                check = set([item[2] for item in random_results])
                if len(check) == 3: break
        else:
            random_results = results
        
        result = '\n'.join([' | '.join(map(str, tpl)) for tpl in random_results])
        return result

In [99]:
# 給使用者輸入表去複製然後傳一次訊息
# 格式
'''
年度：2000>
中文片名：我
原文片名： I
國別：美國
語言：英語
類型 ：Romance
'''
row_msg ='''
年度：
中文片名：
原文片名：
國別：臺灣
語言：
類型 ：
'''

RS = Recommand_System()
request = RS.cleanmsg(row_msg)
print(RS.random_recommand(request))

2018 | 比悲傷更悲傷的故事 | MORE THAN BLUE | 臺灣 | 國語 | Romance
2017 | 海闊天空 | A JOURNEY OF 35 | 臺灣 | 國語 | Drama
2020 | 打噴嚏 | A Choo | 臺灣 | 國語 | Action, Comedy, Fantasy


In [ ]:
# 完全隨機推薦不限制
RS = Recommand_System()
print(RS.random_recommand())